# Open and Crop Landsat RS Data

### Landsat file naming convention

Landsat and many other satellite RS data is named in a way that tells you about:
- When the data were collected and processed
- What sensor was used to collect the data
- What satellite was used to collect the data

There are a few key components of the LS8 collection file name. The 1st scene that you wok with below is named:

LC080340322016072301T1-SC20180214145802

At the start of the name, you see LC08:
- L: Landsat Sensor
- C: OLI/TIRS combined platform
- 08: LS8
- 034032: The next 6 digits represent the path and row of the scene. This identifies the spatial converage of the scene

Finally, you have a date. In your case as follows:
- 20160723: representing the ymd that the data were collected-

The 2nd part oif the file name above tells you more about when the data were last processed.

## Open LS .tif files

In [2]:
import os
from glob import glob # File manipulation
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

# Download data and set work dir
data = et.data.get_data("cold-springs-fire")
os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data"))

You will be working in the landsat-collect dir. Notice that the data in that directory are stored by individual band. Each file is a single geotiff (.tif) rather than 1 tif with all the bands which is what you worked with in the previous lesson with NAIP data.